### Ladi_ai_RAG

- API DOCS : http://43.202.173.71:3000/
- TASK : 루틴 추천
- 루틴 관련 데이터 저장(vectorDB) -> 검색(Retrieve) -> LLM(gpt-4o)의 Context로 사용

## LADI API

### API

### 크롤링 테스트

In [ ]:
import bs4
import re
from langchain_community.document_loaders import WebBaseLoader

# 여러 개의 url 지정 가능
url1 = "https://jhealthmedia.joins.com/article/article_view.asp?pno=28775"

loader = WebBaseLoader(
    web_path=url1,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name={"keyword", "title", "meta"}
        )
    ),
)

# 문서 로드
docs = loader.load()

# 필요한 정보 추출 함수 정의
def filter_document(doc):
    # 메타데이터에서 필요한 정보 추출
    metadata = {}
    if 'title' in doc.metadata:
        metadata['title'] = doc.metadata['title']
    if 'description' in doc.metadata:
        metadata['description'] = doc.metadata['description']
    if 'keywords' in doc.metadata:
        metadata['keywords'] = doc.metadata['keywords']
    if 'meta' in doc.metadata:
        metadata['meta'] = doc.metadata['meta']
    
    # 페이지 내용 정리
    clean_content = re.sub(r'<.*?>', '', doc.page_content)  # HTML 태그 제거
    clean_content = re.sub(r'\s+', ' ', clean_content).strip()  # 불필요한 공백 제거
    
    return {
        'metadata': metadata,
        'content': clean_content
    }

# 필터링된 문서 리스트 생성
filtered_docs = [filter_document(doc) for doc in docs]

# 필터링된 문서 수 출력
print(f"Filtered documents count: {len(filtered_docs)}")

# 필터링된 문서 내용 출력 (예시로 첫 번째 문서)
if filtered_docs:
    print("First filtered document:")
    print(filtered_docs[0])

In [ ]:
filtered_docs



In [94]:
url1 = "https://jhealthmedia.joins.com/article/article_view.asp?pno=28772"

loader = WebBaseLoader(
    web_path=url1,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name="meta"  # 'script' 대신 'meta' 태그를 파싱하도록 변경
        )
    ),
)

# 문서 로드
docs = loader.load()


In [ ]:
docs[0].metadata

In [ ]:
docs[0].page_content

In [ ]:
import requests
from bs4 import BeautifulSoup

url1 = "https://jhealthmedia.joins.com/article/article_view.asp?pno=28772"

# 웹 페이지 가져오기
response = requests.get(url1)
response.encoding = 'utf-8'  # 한글 인코딩 설정

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# keyword meta 태그 찾기
keyword_meta = soup.find('meta', attrs={'name': 'keyword'})
date = soup.find_all('em', attrs={'class': 'mg'})


if keyword_meta:
    keywords = keyword_meta.get('content')
    print("Keywords:", keywords)
else:
    print("Keyword meta tag not found")
print(date)

In [ ]:
import requests
from bs4 import BeautifulSoup

url1 = "https://jhealthmedia.joins.com/article/article_view.asp?pno=28772"

# 웹 페이지 가져오기
response = requests.get(url1)
response.encoding = 'utf-8'  # 한글 인코딩 설정

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# keyword meta 태그 찾기
keyword_meta = soup.find('meta', attrs={'name': 'keyword'})
date_elements = soup.find_all('em', attrs={'class': 'mg'})

# 키워드 추출
if keyword_meta:
    keywords = keyword_meta.get('content')
    print("Keywords:", keywords)
else:
    print("Keyword meta tag not found")

# 날짜 추출
if len(date_elements) > 1:  # 두 번째 요소가 존재하는지 확인
    date = date_elements[1].get_text().split(" ")[1]  # 두 번째 요소의 텍스트 가져오기
    print("Date:", date)
else:
    print("Date not found")

In [ ]:
import bs4
from bs4 import BeautifulSoup

html_doc = """
<html>
<head>
    <meta name="keyword" content="example, test">
    <meta name="description" content="This is a test description.">
    <title>Test Document</title>
</head>
<body>
    <h1>Hello, World!</h1>
</body>
</html>
"""

# SoupStrainer를 사용하여 'meta' 태그만 파싱
strainer = bs4.SoupStrainer(name="meta")
soup = BeautifulSoup(url1, 'html.parser', parse_only=strainer)

# 필터링된 'meta' 태그 출력
for meta in soup.find_all("meta"):
    print(meta)

In [ ]:
import bs4
import re
import requests
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup

def get_metadata_from_url(url):
    # 웹 페이지 가져오기
    response = requests.get(url)
    response.encoding = 'utf-8'  # 한글 인코딩 설정
    
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # keyword meta 태그 찾기
    keyword_meta = soup.find('meta', attrs={'name': 'keyword'})
    date_elements = soup.find_all('em', attrs={'class': 'mg'})
    
    metadata = {}
    
    # 키워드 추출
    if keyword_meta:
        metadata['keywords'] = keyword_meta.get('content')
    
    # 날짜 추출
    if len(date_elements) > 1:
        metadata['date'] = date_elements[1].get_text().split(" ")[1]
    
    return metadata

def filter_document(doc):
    # 메타데이터에서 필요한 정보 추출
    metadata = {}
    if 'title' in doc.metadata:
        metadata['title'] = doc.metadata['title']
    if 'description' in doc.metadata:
        metadata['description'] = doc.metadata['description']
    
    # 페이지 내용 정리
    clean_content = re.sub(r'<.*?>', '', doc.page_content)  # HTML 태그 제거
    clean_content = re.sub(r'\s+', ' ', clean_content).strip()  # 불필요한 공백 제거
    
    # "기자의 최신 글" 이전의 내용 제거
    if "기자의 최신 글" in clean_content:
        clean_content = clean_content.split("기자의 최신 글")[0].strip()
    
    # "기자"라는 단어가 포함된 경우, 그 앞의 내용을 제거
    if "기자" in clean_content:
        clean_content = clean_content.split("기자")[-1].strip()
    
    # "SNS 공유 및 댓글" 이후의 내용 제거
    clean_content = clean_content.replace("닫기 SNS 공유 및 댓글 페이스북 트위터 카카오스토리", "").strip()
    return {
        'metadata': metadata,
        'content': clean_content
    }

# URL 설정
url1 = "https://jhealthmedia.joins.com/article/article_view.asp?pno=28775"

# WebBaseLoader를 사용하여 문서 로드
loader = WebBaseLoader(
    web_path=url1,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name={"meta"}
        )
    ),
)

# 문서 로드
docs = loader.load()
url_list = [{"url": f"https://jhealthmedia.joins.com/article/article_view.asp?pno={pno}", "id": pno} for pno in range(13000, 28776)]
print(url_list)

In [ ]:
import bs4
import re
import requests
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup
from tqdm import tqdm  # tqdm 임포트 추가
import json  # JSON 모듈 추가

def get_metadata_from_url(url):
    # 웹 페이지 가져오기
    response = requests.get(url)
    response.encoding = 'utf-8'  # 한글 인코딩 설정
    
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # keyword meta 태그 찾기
    keyword_meta = soup.find('meta', attrs={'name': 'keyword'})
    date_elements = soup.find_all('em', attrs={'class': 'mg'})
    
    metadata = {}
    
    # 키워드 추출
    if keyword_meta:
        metadata['keywords'] = keyword_meta.get('content')
    
    # 날짜 추출
    if len(date_elements) > 1:
        metadata['date'] = date_elements[1].get_text().split(" ")[1]
    
    return metadata

def filter_document(doc):
    # 메타데이터에서 필요한 정보 추출
    metadata = {}
    if 'title' in doc.metadata:
        metadata['title'] = doc.metadata['title']
    if 'description' in doc.metadata:
        metadata['description'] = doc.metadata['description']
    
    # 페이지 내용 정리
    clean_content = re.sub(r'<.*?>', '', doc.page_content)  # HTML 태그 제거
    clean_content = re.sub(r'\s+', ' ', clean_content).strip()  # 불필요한 공백 제거
    
    # "기자의 최신 글" 이전의 내용 제거
    if "기자의 최신 글" in clean_content:
        clean_content = clean_content.split("기자의 최신 글")[0].strip()
    
    # "기자"라는 단어가 포함된 경우, 그 앞의 내용을 제거
    if "기자" in clean_content:
        clean_content = clean_content.split("기자")[-1].strip()
    
    # "SNS 공유 및 댓글" 이후의 내용 제거
    clean_content = clean_content.replace("닫기 SNS 공유 및 댓글 페이스북 트위터 카카오스토리", "").strip()
    
    return {
        'metadata': metadata,
        'content': clean_content
    }

def convert_to_json(filtered_docs):
    # 필터링된 문서를 JSON 형식으로 변환
    return json.dumps(filtered_docs, ensure_ascii=False, indent=4)

# URL 설정
url_list = [{"url": f"https://jhealthmedia.joins.com/article/article_view.asp?pno={pno}", "id": pno} for pno in range(13000, 28775)]

# 문서 로드 및 필터링된 문서 리스트 생성
filtered_docs = []
for url_info in tqdm(url_list, desc="문서 로드 진행 중", unit="문서"):
    url = url_info['url']
    
    # WebBaseLoader를 사용하여 문서 로드
    loader = WebBaseLoader(
        web_path=url,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                name={"meta"}
            )
        ),
    )
    
    # 문서 로드
    docs = loader.load()
    
    # 추가 메타데이터 가져오기
    additional_metadata = get_metadata_from_url(url)
    
    # 필터링된 문서 리스트 생성
    for doc in docs:
        filtered_doc = filter_document(doc)
        filtered_doc['metadata'].update(additional_metadata)
        filtered_docs.append(filtered_doc)

# 결과 출력
print(f"Filtered documents count: {len(filtered_docs)}")
if filtered_docs:
    print("\nFirst filtered document metadata:")
    print("Title:", filtered_docs[0]['metadata'].get('title', 'No title'))
    print("Keywords:", filtered_docs[0]['metadata'].get('keywords', 'No keywords'))
    print("Date:", filtered_docs[0]['metadata'].get('date', 'No date'))
    print("Description:", filtered_docs[0]['metadata'].get('description', 'No description'))
    print("Content:", filtered_docs[0]['content'])

In [ ]:
# JSON 파일로 저장
import os
output_directory = "/Users/minu/dev/LADI/LADI_ai/data/crawling_data"

# 디렉토리가 존재하지 않으면 생성
os.makedirs(output_directory, exist_ok=True)

# 각 문서에 대해 JSON 파일로 저장
for index, doc in enumerate(filtered_docs):
    output_file_path = os.path.join(output_directory, f"filtered_document_{index + 1}.json")  # 인덱스를 파일 이름에 포함
    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(convert_to_json([doc]))  # 각 문서를 JSON 형식으로 저장

print(f"Filtered documents saved to {output_directory}")

In [ ]:
# JSON으로 변환된 필터링된 문서 출력
json_output = convert_to_json(filtered_docs)
print("\nFiltered documents in JSON format:")
print(json_output)

In [ ]:
len(filtered_docs)


## RAG Setting

### Json_Loader 지식검색대화-가족과주거

In [ ]:
import json
import os

# JSON 파일 로드 함수 정의
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# 특정 디렉토리에서 "가족과 주거"를 포함하는 모든 JSON 파일 로드
def load_family_housing_json_files(directory):
    combined_data = {
        "info": {},
        "utterances": []
    }

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            data = load_json_file(file_path)
            
            # info 데이터 통합 (중복 방지)
            if not combined_data["info"]:
                combined_data["info"] = data["info"]
            
            # utterances 데이터 통합
            combined_data["utterances"].extend(data.get("utterances", []))

    return combined_data

# 디렉토리 경로 설정
directory_path = "data/045.지식검색 대화/01-1.정식개방데이터/Training/02.라벨링데이터/TL_가족과 주거/"

# JSON 파일 로드
final_data = load_family_housing_json_files(directory_path)

# 결과 출력
print(json.dumps(final_data, ensure_ascii=False, indent=4))

In [ ]:
len(final_data)

In [11]:
import os
import json
from glob import glob
from langchain.schema import Document
from tqdm import tqdm  # tqdm 라이브러리 임포트

# 기본 경로 설정
base_dir = "data/045.지식검색 대화/01-1.정식개방데이터/Training/02.라벨링데이터"

# TL_ 로 시작하는 모든 폴더 찾기
tl_folders = [
    "data/045.지식검색 대화/01-1.정식개방데이터/Training/02.라벨링데이터/TL_가족과 주거",
    "data/045.지식검색 대화/01-1.정식개방데이터/Training/02.라벨링데이터/TL_사회와 생활"
]

structured_data = []  # 구조화된 데이터를 저장할 리스트

# 각 폴더에서 json 파일 처리
for folder in tl_folders:
    json_files = glob(os.path.join(folder, "*.json"))
    
    # tqdm을 사용하여 진행률 표시
    for json_file in tqdm(json_files, desc=f"Processing files in {folder}"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # 필요한 메타데이터만 구성
                metadata = {
                    "category": data.get('info', {}).get('category'),
                    "id": data.get('info', {}).get('id'),
                    "name": data.get('info', {}).get('name'),
                    "topic": data.get('info', {}).get('topic'),
                }

                # 콘텐츠 구성 (필요한 필드만 남기기)
                content = []
                for utterance in data.get('utterances', []):
                    content.append({
                        "turn_id": utterance.get('turn_id'),
                        "utterance_id": utterance.get('utterance_id'),
                        "role": utterance.get('role'),
                        "text": utterance.get('text')
                    })

                # 구조화된 데이터 추가
                structured_data.append({
                    "metadata": metadata,
                    "content": content
                })
                
        except json.JSONDecodeError as e:
            print(f"JSON 파일 파싱 중 오류 발생 - {json_file}: {e}")
        except Exception as e:
            print(f"파일 처리 중 오류 발생 - {json_file}: {e}")

# 결과 출력
print(f"총 {len(structured_data)}개의 구조화된 데이터를 로드했습니다.")

# 첫 번째 구조화된 데이터의 메타데이터와 콘텐츠 출력 (확인용)
if structured_data:
    print("\n첫 번째 구조화된 데이터 메타데이터:")
    print(structured_data[0]['metadata'])
    print("\n첫 번째 구조화된 데이터 콘텐츠:")
    print(structured_data[0]['content'])

Processing files in data/045.지식검색 대화/01-1.정식개방데이터/Training/02.라벨링데이터/TL_가족과 주거: 100%|██████████| 1688/1688 [00:00<00:00, 3819.38it/s]
Processing files in data/045.지식검색 대화/01-1.정식개방데이터/Training/02.라벨링데이터/TL_사회와 생활: 100%|██████████| 1395/1395 [00:00<00:00, 4232.84it/s]

총 3083개의 구조화된 데이터를 로드했습니다.

첫 번째 구조화된 데이터 메타데이터:
{'category': '지식검색 대화', 'id': 'BK22008943', 'name': '2022 한국어 블렌더봇 데이터 BK22008943', 'topic': '가족과 주거'}

첫 번째 구조화된 데이터 콘텐츠:
[{'turn_id': 'BK22008943-1', 'utterance_id': 'BK22008943.1', 'role': '질문자', 'text': '아이를 키우는 것은 정말 어렵네요. 처음이다 보니 모든 게 서툴러서 그런 것 같아요. 혹시 제가 몇 가지 여쭤봐도 될까요?'}, {'turn_id': 'BK22008943-1', 'utterance_id': 'BK22008943.2', 'role': '전문가', 'text': '그럼요. 어떤 것이 궁금하신가요? 제가 어린이집을 운영하고 있다 보니 육아에 대해서는 누구보다도 잘 알아요.'}, {'turn_id': 'BK22008943-2', 'utterance_id': 'BK22008943.3', 'role': '질문자', 'text': '너무 든든하네요. 제가 모유 수유를 못해서 아이에게 분유를 급여하고 있는데 혹시 분유는 몇 개월까지 먹이면 되나요?'}, {'turn_id': 'BK22008943-2', 'utterance_id': 'BK22008943.4', 'role': '전문가', 'text': '보통 분유는 두 돌까지 먹인다고 하지만 조금 더 오래 먹는다고 문제가 되지는 않아요. 하지만 보통 돌이 지나면 서서히 신선하고 거의 가공이 하지 않은 일반 우유를 먹이게 되죠. 분유를 먹는 아기는 이유식이나 음식을 조금씩 먹게 되는 만 1살 즉, 돌이 되면 서서히 생우유로 바꾸어 먹이는 것이 좋아요.'}, {'turn_id': 'BK22008943-3', 'utterance_id': 'BK22008943.5', 'role': '질문자', 'text': '그럼 돌까지는 먹여야 하는군요. 아, 그리고 혹시 아기 

In [2]:
print(len(structured_data))
print(type(structured_data))
print(structured_data[0]['metadata'])


3083
<class 'list'>
{'category': '지식검색 대화', 'evaluation': {'speaker1UttAvg_rating': 4.7, 'speaker1UttGrade': '우수', 'speaker2UttAvg_rating': 4.6, 'speaker2UttGrade': '우수'}, 'id': 'BK22008943', 'name': '2022 한국어 블렌더봇 데이터 BK22008943', 'topic': '가족과 주거', 'votes': [{'speaker1UttScore': 5.0, 'speaker2UttScore': 5.0, 'voter_id': 'bk_mc_voter0021'}, {'speaker1UttScore': 4.2, 'speaker2UttScore': 3.8, 'voter_id': 'bk_nhn_voter0008'}, {'speaker1UttScore': 4.6, 'speaker2UttScore': 4.8, 'voter_id': 'bk_mc_voter0020'}, {'speaker1UttScore': 4.6, 'speaker2UttScore': 4.4, 'voter_id': 'bk_nhn_voter0007'}, {'speaker1UttScore': 5.0, 'speaker2UttScore': 5.0, 'voter_id': 'bk_mc_voter0046'}]}


In [3]:
from rag.base import RetrievalChain
from rag.json import JSONRetrievalChain
rag = JSONRetrievalChain(source_uri=structured_data, docs=structured_data)


In [4]:
rag=rag.create_chain(cache_mode='load', local_db="./cached_TL_family_housing and social/", category="건강", mode='kiwi')


로컬에서 벡터 저장소 로드 중


/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='사용자를 위한 개인화된 인사말' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='루틴 단계의 제목' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='세부 실천 단계들' extra={} is no

In [6]:
print(type(rag.retriever))

<class 'rag.base.KiwiBM25Retriever'>


In [9]:
result=rag.retriever.invoke("간 건강")
print(result)

[Document(metadata={'category': '지식검색 대화', 'evaluation': {'speaker1UttAvg_rating': 4.6, 'speaker1UttGrade': '우수', 'speaker2UttAvg_rating': 4.7, 'speaker2UttGrade': '우수'}, 'id': 'BK22004054', 'name': '2022 한국어 블렌더봇 데이터 BK22004054', 'topic': '가족과 주거', 'votes': [{'speaker1UttScore': 4.4, 'speaker2UttScore': 4.8, 'voter_id': 'bk_mc_voter0020'}, {'speaker1UttScore': 4.4, 'speaker2UttScore': 4.0, 'voter_id': 'bk_mc_voter0025'}, {'speaker1UttScore': 4.8, 'speaker2UttScore': 5.0, 'voter_id': 'bk_mc_voter0036'}, {'speaker1UttScore': 4.6, 'speaker2UttScore': 4.8, 'voter_id': 'bk_mc_voter0014'}, {'speaker1UttScore': 4.8, 'speaker2UttScore': 5.0, 'voter_id': 'bk_mc_voter0009'}]}, page_content='내 결혼 앞두고 엄마랑 아빠가 요즘 마음이 싱숭생숭한가봐. 아무래도 말로만 듣던 빈 둥지 증후군 같은데, 어떻게 생각해?\n그럴 수 있지. 보통 자녀가 대학을 입학하거나 취직해서 집을 떠나는 경우에 많은 부모님들이 상실감을 느끼고 슬픔에 빠지시곤 해. 아니면 퇴직을 하시거나 정리해고, 사별, 폐경과 같은 일로 어려움을 겪으시기도 하고. 이와 같은 증상을 빈 둥지 증후군이라고 해.\n역시 그거 맞겠지? 이게 참 결혼을 안 할 수도 없고, 그렇다고 상심한 부모님 모습을 마냥 외면할 수도 없고, 정말 걱정이야. 혹시 이런 증상이 심해지면 다른 쪽으로도 

In [5]:
result=rag.chain.invoke({"context":rag.retriever,"question":"간 건강을 위한 루틴을 짜줘"})

AttributeError: 'list' object has no attribute 'form'

Failed to multipart ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. The content length of 309132323 bytes exceeds the maximum size limit of 20971520 bytes. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')))"))
Content-Length: 309132323
API Key: lsv2_********************************************0ftrace=9f5dfcad-3941-4179-b2a2-620fa17dc0ea,id=fef1706a-c5db-4593-b3c9-d1b540bf64c2; trace=9f5dfcad-3941-4179-b2a2-620fa17dc0ea,id=5c980de9-6908-47b2-a9fc-585d896c342d; trace=9f5dfcad-3941-4179-b2a2-620fa17dc0ea,id=b7eff620-cc9e-4fb4-9b2f-9054fa662ccb; trace=9f5dfcad-3941-4179-b2a2-620fa17dc0ea,id=f7dad86b-8746-4a32-a250-d382919b728e; trace=9f5dfcad-3941-4179-b2a2-620fa17dc0ea,id=17a6e6ab-e314-45e9-9dff-e99e5ebf4bd

In [10]:
# 1. 벡터 DB에 저장된 문서 확인
def check_documents(retriever):
    print("저장된 문서 목록:")
    for doc in retriever.docs:
        print(f"ID: {doc.metadata.get('id')}, Topic: {doc.metadata.get('topic')}")
        print(f"Content: {doc.page_content[:100]}...")  # 내용의 일부만 출력
        print("-" * 50)

# 2. 리트리버의 검색 결과 확인
def test_retriever_query(retriever, query):
    print(f"'{query}'에 대한 검색 결과:")
    results = retriever.invoke(query)
    for i, doc in enumerate(results):
        print(f"Result {i + 1}:")
        print(f"ID: {doc.metadata.get('id')}, Topic: {doc.metadata.get('topic')}")
        print(f"Content: {doc.page_content[:100]}...")  # 내용의 일부만 출력
        print("-" * 50)

# 사용 예시
# 1. 리트리버의 문서 확인
check_documents(rag.retriever)

# 2. 리트리버에 간 건강에 대한 질문 테스트
test_retriever_query(rag.retriever, "간 건강")

저장된 문서 목록:
ID: BK22008943, Topic: 가족과 주거
Content: 아이를 키우는 것은 정말 어렵네요. 처음이다 보니 모든 게 서툴러서 그런 것 같아요. 혹시 제가 몇 가지 여쭤봐도 될까요?
그럼요. 어떤 것이 궁금하신가요? 제가 어린이집을 운영하...
--------------------------------------------------
ID: BK22007961, Topic: 가족과 주거
Content: 내 친구 이번에 쌍둥이를 임신했대. 친구 아버지가 쌍둥이신데 이것도 유전인가?
쌍둥이 임신은 가족력이 어느 정도 반영된 거라고 볼 수도 있는데 사실 어느 대에 영향을 받아서 태어나...
--------------------------------------------------
ID: BK22011967, Topic: 가족과 주거
Content: 얼마 전에 위층에 새로운 가족이 이사를 왔는데 쿵쿵거리는 발소리 때문에 매일이 너무 힘들어요.
층간 소음 때문에 많이 힘드시군요. 가장 편안해야 할 집에서 오히려 스트레스를 받게 ...
--------------------------------------------------
ID: BK22002628, Topic: 가족과 주거
Content: 2세 계획을 세우고 있는데 학비가 장난이 아니라는 말을 많이 들었어요. 그래서 미리 유치원이나 교육비도 좀 알아봐야 할 것 같아요.
미리 계획을 세우면 좋죠. 아기가 크기 전에 계...
--------------------------------------------------
ID: BK22015301, Topic: 가족과 주거
Content: 엊그제 처음으로 아이를 소아치과에 데려가 치아 검진을 받았어요. 입안을 잘 살펴본 적이 없어서 몰랐는데 앞니에 우식증이 조금 생겼다고 하더라구요. 우식증이 뭐죠?
치아 우식증은 영...
--------------------------------------------------
ID: BK22006878, Top

In [ ]:
from langchain.schema import Document

# structured_data를 Document 객체 리스트로 변환
from tqdm import tqdm  # tqdm 라이브러리 임포트

documents = [
    Document(
        page_content=str(data['content']),  # 'content' 필드에서 페이지 내용을 문자열로 변환합니다.
        metadata=data['metadata']  # 'metadata' 필드에서 메타데이터를 가져옵니다.
    )
    for data in tqdm(structured_data, desc="문서 변환 진행 중")  # tqdm을 사용하여 진행 상황 표시
]

# 리트리버 생성
rag_retriever = rag.create_retriever(documents, category="건강", mode='kiwi')
print(rag_retriever.invoke("가족과 주거에 대해 설명해줘"))

In [11]:
user_info={
  "USER_ID": 123,
  "NAME": "이상하",
  "age": 23,
  "GOAL": "건강한 몸 만들기",
  "gender": "남성",
  "JOB": "학생",
  "weight": 65,
  "height": 168,
  "workout_frequency": "1회 미만",
  "workout_location": "집",
  "category": "건강",
  "goal": "건강한 몸 만들기",
  "selected_tasks": ["식습관", "수면습관", "운동", "기타"],
  "start_time": "07:00",
  "repeat_days": ["월", "수", "금"],
  "notification": "True"
}

In [ ]:
rag_retriever = rag.create_retriever(structured_data,category="건강",mode='kiwi')
print(rag_retriever.invoke("가족과 주거에 대해 설명해줘"))


In [ ]:
question = "새해 기념 루틴을 짜워!!"
result = rag.invoke({
    "context": rag_retriever,
    "question": question,
    "USER_ID": user_info.get("USER_ID"),
    "NAME": user_info.get("NAME"),
    "age": user_info.get("age"),
    "GOAL": user_info.get("GOAL"),
    "gender": user_info.get("gender"),
    "JOB": user_info.get("JOB"),
    "weight": user_info.get("weight"),
    "height": user_info.get("height"),
    "workout_frequency": user_info.get("workout_frequency"),
    "workout_location": user_info.get("workout_location"),
    "category": user_info.get("category"),
    "goal": user_info.get("goal"),
    "selected_tasks": user_info.get("selected_tasks"),
    "start_time": user_info.get("start_time"),
    "repeat_days": user_info.get("repeat_days"),
    "notification": user_info.get("notification")
})
print(result)

In [ ]:
# result를 md 형태로 출력
print(f"# 인사말\n{result.greeting}\n")
print(f"# 루틴\n{result.routines}\n")
print(f"# 팁\n{result.tips}\n")

In [ ]:
from fastapi import FastAPI
import httpx
import nest_asyncio

nest_asyncio.apply()  # 이벤트 루프 문제 해결

app = FastAPI()

@app.post("/routine/")
async def create_routine(user_info: dict, question: str):
    result = rag.chain.invoke({
        "context": rag_retriever,
        "question": question,  # POST 요청에서 받은 question 사용
        "USER_ID": user_info.get("USER_ID"),
        "NAME": user_info.get("NAME"),
        "age": user_info.get("age"),
        "GOAL": user_info.get("GOAL"),
        "gender": user_info.get("gender"),
        "JOB": user_info.get("JOB"),
        "weight": user_info.get("weight"),
        "height": user_info.get("height"),
        "workout_frequency": user_info.get("workout_frequency"),
        "workout_location": user_info.get("workout_location"),
        "category": user_info.get("category"),
        "goal": user_info.get("goal"),
        "selected_tasks": user_info.get("selected_tasks"),
        "start_time": user_info.get("start_time"),
        "repeat_days": user_info.get("repeat_days"),
        "notification": user_info.get("notification")
    })
    
    # API 호출 예시
    async with httpx.AsyncClient() as client:
        api_response = await client.post("http://localhost:8000/routine", json={
            "greeting": result.greeting,
            "routines": [routine.dict() for routine in result.routines],  # RoutineStep을 dict로 변환
            "tips": result.tips
        })
    
    # API 응답의 원본 텍스트 출력
    print(api_response.text)  # 응답의 원본 텍스트 출력

    try:
        api_response_json = api_response.json()
    except ValueError:
        api_response_json = {"error": "Invalid JSON response", "response_text": api_response.text}

    return {
        "greeting": result.greeting,
        "routines": [routine.dict() for routine in result.routines],  # RoutineStep을 dict로 변환
        "tips": result.tips,
        "api_response": api_response_json  # API 응답 추가
    }

import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [13]:

from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


### Json_Loader

In [ ]:
import json
from langchain.schema import Document
from pprint import pprint

# JSON 파일 로드 함수 정의
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 데이터 로드
data = load_json_file("data/dummy_data.json")
\
# 필요한 정보 추출 및 Document 객체 생성
docs = [
    Document(
        page_content=item["context"],
        metadata={
            "id": item["id"],
            "question": item["question"],
            "age": item["age"],
            "gender": item["gender"],
            "Job": item["Job"],
            "Difficult": item["Difficult"],
            "goal": item["goal"]
        }
    )
    for item in data
]

# 결과 출력
pprint(docs)

### PDF_Loader

In [1]:
from rag.pdf import PDFRetrievalChain

pdf_chain = PDFRetrievalChain(["data/세계보건기구(WHO) 신체활동 권장지침.pdf"])
pdf_chain.create_embedding()  # cached_embeddings 초기화
pdf = pdf_chain.create_chain()
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

In [ ]:
pdf_retriever.invoke("신체활동 권장지침에 대해 설명해줘")

In [ ]:
pdf_chain.invoke({"context":pdf_retriever,"question":"신체활동 권장지침에 대해 설명해줘"})


### Embedding and Vectorstore

In [25]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# 1. 벡터 저장소 생성
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")


fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    passage_embeddings, fs, namespace=passage_embeddings.model
)
vectorstore = FAISS(embedding_function=cached_embeddings)


### text_spliting and add_documents

In [ ]:


# 2. 문서 임베딩 및 저장
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
vectorstore.add_documents(texts)


### 더미 데이터 생성

### generate one

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional

class SubRoutine(BaseModel):
    """운동 루틴을 추천합니다."""
    emoji: str
    routineDetail: str
    minuteDuration: str

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0,response_format=SubRoutine)
response = llm.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")
# structured_llm = llm.with_structured_output(
#     SubRoutine,
#     method="json_schema",
#     include_raw=False,
#     strict=True
# )

# response = structured_llm.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")
import json

content = json.loads(response.content)
json_response = json.dumps(content, indent=2, ensure_ascii=False)
print(json_response)
print(type(json_response))


### LLM Answer

In [33]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate

# 5. RAG 파이프라인 구축
prompt_template = PromptTemplate(
    input_variables=["context", "question", "user_id", "age", "gender", "height", "weight", "goal_id", "frequency_id", "location_id", "start_time", "end_time", "repeat_days"],
        template="""당신은 API 서버입니다.
{context}를 참고하여 사용자에 맞는 루틴을 추천해주세요.

사용자 정보:
- 사용자 ID: {user_id}
- 나이: {age}
- 성별: {gender} 
- 신장: {height}cm
- 체중: {weight}kg
- 운동 목표 ID: {goal_id}
- 운동 빈도 ID: {frequency_id}
- 선호 운동 장소 ID: {location_id}
- 운동 시작 시간: {start_time}
- 운동 종료 시간: {end_time}
- 반복 요일: {repeat_days}

질문: {question}
답변:
응답 형식
{{
  "subRoutine": {{
      "emoji": "이모지",
      "routineDetail": "루틴 설명", 
      "minuteDuration": "시간(분 단위)"
  }}
}}
"""
)

retriever = vectorstore.as_retriever()

class SubRoutine(BaseModel):
    """운동 루틴을 추천합니다."""
    emoji: str = Field(description="이모지")
    routineDetail: str = Field(description="루틴 설명")
    minuteDuration: str = Field(description="시간(분 단위)")

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.7)
parser = JsonOutputParser(pydantic_object=SubRoutine)
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

retrieved_docs = retriever.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")


In [ ]:

# RAG 체인 생성
from operator import itemgetter
chain = (
    {
        "question":itemgetter("question"),
        "context":itemgetter("context"),
        "age":itemgetter("age"),
        "gender":itemgetter("gender"),
        "job":itemgetter("job"),
        "difficult":itemgetter("difficult"),
        "goal":itemgetter("goal")
    }
    | prompt_template
    | llm
    | parser
)

# 결과 생성
results = []
for i in range(5):

    result = chain.invoke(
        {"context":retrieved_docs,"question":"시험 공부를 위한 목표 달성 루틴 생성해줘",
         "age":itemgetter("age"),"gender":itemgetter("gender"),
         "job":itemgetter("job"),"difficult":itemgetter("difficult"),
         "goal":itemgetter("goal")}
    )
    results.append(result)

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result)
    print()

In [43]:
import random
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage


# 프롬프트 템플릿 수정
prompt_template = PromptTemplate(
    input_variables=["context", "question", "age", "gender", "job", "difficult", "goal", "emojis"],
    template="""당신은 API 서버입니다.
{context}를 참고하여 사용자에 맞는 루틴을 추천해주세요.
사용자의 나이는 {age}입니다.
사용자의 성별은 {gender}입니다.
사용자의 직업은 {job}입니다.
사용자의 겪고 있는 어려움은 {difficult}입니다.
사용자의 목표는 {goal}입니다.

가능한 한 다양하고 창의적인 루틴을 생성해 주세요. 루틴의 내용에 맞는 적절한 이모지를 응답에 포함해주세요.

질문: {question}
답변:
응답 형식
{{
  "subRoutine": [
    {{
      "emoji": "이모지",
      "routineDetail": "루틴 설명",
      "minuteDuration": 시간(분 단위)
    }},
    ...
  ]
}}
"""
)

# LLM 설정 (온도 증가 및 타임아웃 설정)
llm = ChatOpenAI(model="gpt-4o", temperature=0.9, request_timeout=30)

# 파서 설정
class SubRoutineItem(BaseModel):
    emoji: str = Field(description="이모지")
    routineDetail: str = Field(description="루틴 설명")
    minuteDuration: int = Field(description="시간(분 단위)")

class SubRoutine(BaseModel):
    subRoutine: List[SubRoutineItem]

parser = JsonOutputParser(pydantic_object=SubRoutine)

# RAG 체인 생성
def create_chain():
    def chain(input_dict):
        prompt = prompt_template.format(
            context=input_dict["context"],
            question=input_dict["question"],
            age=input_dict["age"],
            gender=input_dict["gender"],
            job=input_dict["job"],
            difficult=input_dict["difficult"],
            goal=input_dict["goal"]
        )
        try:
            response = llm([HumanMessage(content=prompt)])
            if isinstance(response.content, str):
                parsed_response = parser.parse(response.content)
                return parsed_response
            else:
                print(f"Unexpected response type: {type(response.content)}")
                return None
        except Exception as e:
            print(f"Error in chain: {e}")
            return None
    return chain

chain = create_chain()

# 결과 생성
# 기존에 작성된 부분은 그대로 유지합니다.

# ...


In [ ]:

# 입력 데이터 리스트 생성
input_datas = [
    {
        "context": retrieved_docs,
        "question": "요즘 피곤해",
        "age": "20",
        "gender": "남성",
        "job": "학생",
        "difficult": "공부 집중력 부족",
        "goal": "시험에서 좋은 성적 받기"
    },
    {
        "context": retrieved_docs,
        "question": "공부에 집중이 안 돼",
        "age": "21",
        "gender": "남성",
        "job": "학생",
        "difficult": "시험 스트레스",
        "goal": "성적 향상"
    },
    {
        "context": retrieved_docs,
        "question": "시간 관리가 어려워",
        "age": "22",
        "gender": "남성",
        "job": "학생",
        "difficult": "시간 부족",
        "goal": "효율적인 공부"
    },
    {
        "context": retrieved_docs,
        "question": "시험 준비 어떻게 해야 할까?",
        "age": "23",
        "gender": "남성",
        "job": "학생",
        "difficult": "공부 방법 몰라",
        "goal": "시험 합격"
    },
    {
        "context": retrieved_docs,
        "question": "집중력이 떨어져",
        "age": "24",
        "gender": "남성",
        "job": "학생",
        "difficult": "집중력 부족",
        "goal": "공부 효율 향상"
    }
]

# 결과 생성
results = []
for i, input_data in enumerate(input_datas, 1):
    try:
        result = chain(input_data)
        if result:
            results.append(result)
    except Exception as e:
        print(f"Error at case {i}: {e}")
        continue

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result)
    print()


In [ ]:

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i} {input_datas[i-1]['question']}:")
    print(result)
    print()